In [47]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [25]:
dtypes = pd.Series({'idaviso': np.dtype('uint64'), 'idpostulante': np.dtype('object'),
                   'se_postulo': np.dtype('uint8'), 'sexo_numerico': np.dtype('uint8'),
                   'edad': np.dtype('uint16'), 'estado_code': np.dtype('uint8'),
                   'sexo_code': np.dtype('uint8'), 'nombre_code': np.dtype('uint8'),
                   'nombre_area_code': np.dtype('uint8'),
                    'denominacion_empresa_code': np.dtype('uint16'),
                   'nivel_laboral_code': np.dtype('uint8'),
                   'tipo_de_trabajo_code': np.dtype('uint8'),
                   'nombre_zona_code': np.dtype('uint8')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))


trainingSet = pd.read_csv('Data/trainingSet.csv',dtype=column_types,low_memory=True)
trainingSet.drop(columns=['Unnamed: 0'],inplace=True)

In [26]:
trainingSet.head()

,idaviso,idpostulante,se_postulo,edad,estado_code,sexo_code,nombre_code,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112248724,NjlD,1,36,2,0,6,120,2468,4,1,7
1,1112286523,ZaO5,1,39,0,0,6,159,952,4,1,7
2,1112272060,ZaO5,1,39,0,0,6,153,36,4,1,7
3,1112288401,ZaO5,1,39,0,0,6,153,3629,4,1,7
4,1112300563,ZaO5,1,39,0,0,6,159,350,4,1,7


In [27]:
trainingSet.info()

#TODO: ENCODEAR TESTING Y TRAINING SETS

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9439964 entries, 0 to 9439963
Data columns (total 12 columns):
idaviso                      uint64
idpostulante                 object
se_postulo                   uint8
edad                         uint16
estado_code                  uint8
sexo_code                    uint8
nombre_code                  uint8
nombre_area_code             uint8
denominacion_empresa_code    uint16
nivel_laboral_code           uint8
tipo_de_trabajo_code         uint8
nombre_zona_code             uint8
dtypes: object(1), uint16(2), uint64(1), uint8(8)
memory usage: 252.1+ MB


# Inicializo el testing set

In [28]:
testingSet = pd.read_csv('TestingSets/testingSet_imp_mean.csv')
testingSet = testingSet.drop(columns='Unnamed: 0')

In [29]:
testingSet.head()

,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code,edad,estado_code,sexo_code,nombre_code,id,idaviso,idpostulante
0,30.0,465.0,1.0,1.0,1.0,41.0,2.0,0.0,0.0,0,739260,6M9ZQR
1,30.0,465.0,1.0,1.0,1.0,30.0,2.0,1.0,5.0,1,739260,6v1xdL
2,30.0,465.0,1.0,1.0,1.0,35.0,2.0,0.0,6.0,2,739260,ezRKm9
3,170.0,3891.0,3.0,1.0,1.0,68.0,2.0,1.0,3.0,3,758580,1Q35ej
4,170.0,3891.0,3.0,1.0,1.0,31.0,1.0,0.0,6.0,4,758580,EAN4J6


TO DO

# Encoding de sets

In [30]:
lb_postulante = LabelEncoder()
lb_postulante.fit(trainingSet["idpostulante"])

LabelEncoder()

In [31]:
trainingSet["idpostulante"] = lb_postulante.transform(trainingSet['idpostulante'])

In [ ]:
testingSet["idpostulante"] = lb_postulante.transform(testingSet['idpostulante'])

In [32]:
trainingSet.head()

,idaviso,idpostulante,se_postulo,edad,estado_code,sexo_code,nombre_code,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112248724,186038,1,36,2,0,6,120,2468,4,1,7
1,1112286523,266562,1,39,0,0,6,159,952,4,1,7
2,1112272060,266562,1,39,0,0,6,153,36,4,1,7
3,1112288401,266562,1,39,0,0,6,153,3629,4,1,7
4,1112300563,266562,1,39,0,0,6,159,350,4,1,7


# Divido el set de training para aportar la info correspondiente al testing set

# Prueba AdaBoost

In [35]:
clf = AdaBoostClassifier(learning_rate=2.0,n_estimators=60)

In [39]:
y_training = trainingSet['se_postulo']

In [36]:
trainingSet2 = trainingSet.drop(columns=['se_postulo']);

In [37]:
trainingSet2;

In [40]:
clf.fit(trainingSet2, y_training)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [45]:
clf.score(trainingSet2,y_training)

0.7696278290891787

In [52]:
mse = np.mean(cross_val_score(clf, trainingSet2,y_training,scoring='neg_mean_squared_error', cv=4,  n_jobs=2))

In [53]:
mse

-0.24437379665662057

In [41]:
pred = clf.predict(testingSet2)

ValueError: could not convert string to float: '8R48Ex'